In [35]:
import pandas as pd
from reco_systems.CF_knn import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
folder = "database_cleaned"
avis_clean = pd.read_csv(f"database_cleaned/avis_clean.csv", index_col=0)
jeux_clean = pd.read_csv(f"database_cleaned/jeux_clean.csv", index_col=0)

In [25]:
rev_filter, table_assoc = filter_games(df_reviews=avis_clean, min_reviews=10) 
table_assoc.values

array([    4,     6,    15, ..., 10522, 10531, 10546], shape=(2887,))

In [26]:
matrix_ratings = get_matrix_user_game(rev_filter)
matrix_ratings.toarray()

array([[0. , 0. , 8.6, ..., 0. , 0. , 0. ],
       [0. , 0. , 6.5, ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 6. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]], shape=(10085, 2887))

In [27]:
similarity_matrix = calc_similarity_matrix(matrix_ratings, dist_type="cos")

In [28]:
similar_users = get_KNN(similarity_matrix, k=20, user_ind=5)
similar_users

array([ 296, 1286,  327,  738,  739,    5,  498,  492,  740,  741,  743,
         23, 1452, 5189,  676,  376, 1227, 5157,   46,  167])

In [36]:
games_ind = get_games_ind(matrix_ratings, similar_users, n=10)
games_ind

TypeError: get_games_ind() missing 1 required positional argument: 'selected_users'

In [30]:
get_games_df(jeux_clean, table_assoc, games_ind)

,Game id,Game name website,Game name UI,Game name year,Description,Type,Game duration,Min number of players,Max number of players,Age min,Age max
362,362,alchimistes,Alchimistes,Alchimistes (2015),Le 1er au classement BGG à Essen 2014 et le 4è...,Enchères|Bluff|Déduction|Médiéval-fantastique|...,90.0,2.0,4.0,13.0,99.0
1471,1471,carson-city,Carson City,Carson City (2009),"""Carson City"" est un jeu de développement « ur...",Far-west|Placement d'ouvrier,90.0,2.0,5.0,12.0,99.0
3062,3062,evolution-3,Evolution,Evolution (2016),"Survivre ou périr ! Dans Evolution, vous cherc...",Animaux|Choix simultanés|Combinaison|Gestion d...,60.0,2.0,6.0,12.0,99.0
4290,4290,its-a-wonderful-world,It's a Wonderful World,It's a Wonderful World (2019),"Dans It's a Wonderful World, les joueurs sont ...",Draft|Gestion|Science fiction|Programmation,45.0,1.0,5.0,14.0,99.0
5159,5159,le-dilemme-du-roi,Le Dilemme du Roi,Le Dilemme du Roi (2020),Vos choix forgent le destin du royaume ! Le Di...,Diplomatie|Médiéval|Majorité,60.0,3.0,5.0,14.0,99.0
5918,5918,lords-of-waterdeep,Lords of Waterdeep,Lords of Waterdeep (2012),"Waterdeep, la cité des splendeurs - le plus re...",Médiéval-fantastique|Gestion|Placement d'ouvrier,60.0,2.0,5.0,12.0,99.0
6179,6179,mariposas,Mariposas,Mariposas (2020),"Chaque année, au printemps, des millions de pa...",NaN,60.0,2.0,5.0,14.0,99.0
6615,6615,museum,Museum,Museum (2018),Bienvenue dans le monde de Museum! Crée par Ol...,Amateur|Antiquité|Art|Jeux de plateau|Collecti...,90.0,2.0,4.0,10.0,99.0
6705,6705,nagaraja-1,Nagaraja,Nagaraja (2019),"En Inde, les temples jumeaux de deux divinités...","Choix simultanés|Contes, Mythes & Légendes|Par...",30.0,2.0,2.0,9.0,99.0
7899,7899,robinson-crusoe-aventures-sur-lile-maudite,Robinson Crusoé - Aventures sur l’Île Maudite,Robinson Crusoé - Aventures sur l’Île Maudite ...,"Dans Robinson Crusoé, vous et vos amis, incarn...",Points d'action|Ambiance|Card-driven|Coopérati...,120.0,1.0,4.0,14.0,99.0
